# OpenAI Configuration Options

OpenAI has defined some settings you can use to change the behavior of the Large Language Model's response in both subtle and less subtle ways. Whether you aren't getting the response you expected or you think you can maybe get a slightly better response, these configuration options may be a good place to look.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"

#!import ../config/SettingsHelper.cs

using Microsoft.SemanticKernel;

// load settings specific to you!
MySettings settings = Settings.LoadFromFile();

// Configure AI backend used by the kernel
KernelBuilder builder = new();
if (settings.Type == "azure")
    builder.WithAzureTextCompletionService(settings.AzureOpenAI.CompletionsDeployment, settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
else
    builder.WithOpenAITextCompletionService(settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
IKernel kernel = builder.Build();

You'll first need to create a PromptTemplateConfig object. Within that object, you'll need to define the Completion property and it has all of the configuration options for changing the completion generated. These options include: 
- **MaxTokens** This option is of critical importance for three reasons:
-- each LLM model has a maximum number of tokens that can be used per API request. When estimating tokens, think 1 token is approximately 4 characters in English words. For a GPT model, the tokens in the prompt plus the tokens in the generated completion must be less than your chosen model's maximum token length. GPT-3 models are limited to 2,049 tokens, but GPT-4 can go all the way up to 32,768 tokens! Read more about models and their differences [here](https://platform.openai.com/docs/models).
-- OpenAI also has a rate limit. This means you can only make so many requests per tokens every minute. Read more [here](https://platform.openai.com/docs/guides/rate-limits/overview).
-- the number of tokens generated by OpenAI directly corresponds to the monetary cost to you! Read more [here](https://openai.com/pricing). 
- **Temperature** This can be set as low as 0.0 or as high as 2.0. A higher temperature allows the model to generate more "creative" completions. A lower temperature has the opposite effect and completions are more deterministic (though not entirely deterministic!) When generating creative works, you probably want a higher temperature, but when converting one language to another, you probably want a lower temperature.
- **TopP** Unlike Temperature, the range of this setting is 0.0 to 1.0. When the model generates possible options for the next token, TopP limits the number of the likeliest options. In practice, TopP has a similar "randomizing" effect as Temperature. Both settings can be used together.
- **FrequencyPenalty** This can be set as low as 0.0 or as high as 2.0. Setting this value higher discourages the model from repeating words in the generated completion.
- **PresencePenalty** This can be set as low as 0.0 or as high as 2.0. Setting this value higher encourages the model to generate tokens not already generated previously. Though similar to the frequency penalty, note the difference between frequency of words and presence of tokens - where tokens are often parts of words.
- **StopSequences** Stop sequences are a list of characters like "\n" or "###" though you can define anything you want here. This setting can be a little confusing until you play with it. For some completions, the LLM will generate tokens until it hits max tokens, but if you want it to stop generating tokens at a certain point, you can enter stop sequences in your prompt to control this.

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.SemanticFunctions;

const string skPrompt = @"Write a story about: {{$input}}";

PromptTemplateConfig promptConfig = new()
{
    Completion =
    {
        MaxTokens = 300, // each LLM model has a different token limit
        Temperature = 0.7, // 0.0 - 2.0
        TopP = 0.5, // 0.0 - 1.0 top probability to use
        FrequencyPenalty = 0.0, // 0.0 - 2.0
        PresencePenalty = 0.0, // 0.0 - 2.0
        StopSequences = new() { "###" }, // stop generating text when this sequence is generated
    }
};

// register your semantic function
PromptTemplate promptTemplate = new(skPrompt, promptConfig, kernel);
SemanticFunctionConfig functionConfig = new(promptConfig, promptTemplate);
var chatFunction = kernel.RegisterSemanticFunction("ChatBot", "Chat", functionConfig);

string input = "an android on the run from a human";
SKContext completion = await chatFunction.InvokeAsync(input);

Console.WriteLine(completion);

Hopefully running the code above generated an interesting reading break for you!

We begin with a very simple creative writing prompt. We create a PromptTemplateConfig object and set it's Completion property where each of the previously described configuration options resides. We then have to tie it all together before we register everything as a semantic function with the kernel. (NOTE: The Chat function of the ChatBot skill is a built-in function.)

The above code used an inline function, but this tutorial contains a semantic function as well. You can expand the AutomationSkill\MakeAListFunction folders and find the config.json file there that defines the same configuration options using that style of skill. 

# Exercise

Modify the configuration options above to see how they affect the generated completion. Here are some tips, but feel free to explore: 
- Don't raise the Max Tokens and risk hitting rate limits or spending a lot of money. Instead, lower max tokens to see how doing so changes the completion. (A max tokens of 200 or so may be a good number to test repeated runs in this exercise.)
- Play with Temperature and TopP to see how "creative" the completions get! Then set both options to zero and run the same code twice. Does the result surprise you?
- Maximize Frequency Penalty first and see how often the words 'android' and 'human' come up. How does the AI stil generate a story with these penalties in place?
- Since Stop Sequences don't make a lot of sense in this simple creative writing scenario, we'll explore them in our tutorial on prompt engineering so don't worry about playing with them here. :)